In [ ]:
#import libraries
import os
import pandas as pd
import re
import csv
from io import StringIO
from pathlib import Path
import numpy as np
import glob

In [ ]:
#another way to read multiple csvs and add date column
path = r'/Users/gio/Google Drive/Money/House Expenses/Orders Freshmarket' # use your path
all_files = glob.glob(path + "*.csv")

file_date = re.compile('\d{4}-\d{2}-\d{2}')

dfs = []
for file in all_files:
    df = pd.read_csv(file)
    df['Date'] = file_date.findall(file)[0]    
    dfs.append(df)
    
df = pd.concat(dfs, ignore_index=True)


In [ ]:
#get rid of unnecessary rows
bad_items = ['Scroll for more items', 'Description', 'Subtotal','Shipping']
df = df.loc[-df['Product image'].isin(bad_items),:]

In [ ]:
#get rid of Product Image column
df = df.drop(columns = 'Product image')

In [ ]:
#get rid of Pesos
df.Price = df.Price.str.replace('₱','')

In [ ]:
#extract content from description
df['Content'] = df.Description.str.extract(r'(\d+k?g?)', expand = False).str.extract(r'(\d+)').fillna(1)

In [ ]:
#extract units from content
df['Units'] = df.Description.str.extract(r'(\d+k?g?)', expand = False)
df['Units'] = df.Units.str.replace(r'(\d+)','').fillna('pcs')
df['Units'] = df.Units.replace(r'^\s*$',np.NaN, regex = True).fillna('pcs')

In [ ]:
#create item column containing basic item identifier
#remove Fresh
df['Item'] = df.Description.str.replace('Fresh ','')

In [ ]:
#get rid of extra spaces
df.Item = df.Item.str.replace(r'\".+\"','')

In [ ]:
#get rid of extra spaces
pattern = r'(\(.+\))'
df.Item = df.Item.str.replace(pattern,'')

In [ ]:
#get rid of units stuck with texts
pattern = r'(\d+k?g?)'
df.Item = df.Item.str.replace(pattern,'')

In [ ]:
#get rid of Fruits
pattern = 'Fruits '
df.Item = df.Item.str.replace(pattern,'')

In [ ]:
#change types
for col in ['Price', 'Content']:
    df[col] = df[col].astype('float64')

df.Date = df.Date.astype('datetime64')
df.Units = df.Units.astype('category')

In [ ]:
df.Item = df.Item.str.lower()

In [ ]:
#export to csv
#filename_csv_full = 'for import ' + filename.name
#export_csv = df.to_csv (filename_csv_full, index = None, header=True)
df.to_csv('/Users/gio/Google Drive/Money/House Expenses/Freshmarket Logs.csv', index = None, header=True)

In [ ]:
#food_categories = ['Pork','Fruits','Fish','Beef','Chicken']
#df.item.str.contains